In [1]:
%pylab inline
%load_ext autoreload
%autoreload 1

import gc
import fpfs
import fitsio
import numpy as np
from fpfs import simutil
import matplotlib.pylab as plt
import numpy.lib.recfunctions as rfn

%aimport pdet
import pltutil

Populating the interactive namespace from numpy and matplotlib
Do not have lsst pipeline!


In [2]:
ishear =1
igroup =0
do_plot=False
ngrid  =64
ngal=  10
img1=  simutil.make_basic_sim('basicCenter_psf60','g%d-0000' %ishear,0,ny=ngal,nx=ngal,do_write=False)
ngrid2=ngrid*ngal
gsigma =   6.*2.*np.pi/64


# PSF
psfname =   os.path.join(os.environ['homeWrk'],'FPFS2','sim/galaxy_basic_psf60/psf-60.fits')
psf     =   fitsio.read(psfname)
npad    =   (ngrid2-psf.shape[0])//2
psfData =   np.pad(psf,(npad+1,npad),mode='constant')
assert psfData.shape[0]==ngrid2

indX    =   np.arange(32,64*ngal,64)
indY    =   np.arange(32,64*ngal,64)
inds    =   np.meshgrid(indY,indX,indexing='ij')

coords  =   np.array(np.zeros(inds[0].size),dtype=[('pdet_y','i4'),('pdet_x','i4')])
coords['pdet_y']=   np.ravel(inds[0])
coords['pdet_x']=   np.ravel(inds[1])
del indX,indY,inds

In [3]:
out0    =   pdet.get_shear_response(img1,psfData,gsigma=gsigma,\
            coords=coords)
out1    =   pdet.get_shear_response_rfft(img1,psfData,gsigma=gsigma,\
            coords=coords)
for j in range(1,4):
    for i in range(1,4):
        np.testing.assert_almost_equal(np.max(out1['pdet_v%d%dr1' %(j,i)]-out0['pdet_v%d%dr1' %(j,i)]),0,6)
        np.testing.assert_almost_equal(np.max(out1['pdet_v%d%dr2' %(j,i)]-out0['pdet_v%d%dr2' %(j,i)]),0,6)
        np.testing.assert_almost_equal(np.max(out1['pdet_v%d%d' %(j,i)]-out0['pdet_v%d%d' %(j,i)]),0,6)
gc.collect()

used memory: 0.199
used memory: 0.199
used memory: 0.171
used memory: 0.171


0

In [4]:
%%time
for _ in range(10):
    out0    =   pdet.get_shear_response(img1,psfData,gsigma=gsigma,\
            coords=coords)

used memory: 0.199
used memory: 0.199
used memory: 0.199
used memory: 0.199
used memory: 0.199
used memory: 0.199
used memory: 0.199
used memory: 0.199
used memory: 0.199
used memory: 0.199
used memory: 0.199
used memory: 0.199
used memory: 0.199
used memory: 0.199
used memory: 0.199
used memory: 0.199
used memory: 0.199
used memory: 0.199
used memory: 0.199
used memory: 0.199
CPU times: user 1.18 s, sys: 222 ms, total: 1.4 s
Wall time: 1.39 s


In [5]:
%%time
for _ in range(10):
    out1    =   pdet.get_shear_response_rfft(img1,psfData,gsigma=gsigma,coords=coords)

used memory: 0.171
used memory: 0.171
used memory: 0.171
used memory: 0.171
used memory: 0.171
used memory: 0.171
used memory: 0.171
used memory: 0.171
used memory: 0.171
used memory: 0.171
used memory: 0.171
used memory: 0.171
used memory: 0.171
used memory: 0.171
used memory: 0.171
used memory: 0.171
used memory: 0.171
used memory: 0.171
used memory: 0.171
used memory: 0.171
CPU times: user 827 ms, sys: 19.3 ms, total: 846 ms
Wall time: 841 ms


In [10]:
%%time
pdet.test_numba(100)

CPU times: user 5 µs, sys: 1 µs, total: 6 µs
Wall time: 7.87 µs


100

In [9]:
%%time
pdet.test_numba_njit(100)

CPU times: user 37.6 ms, sys: 126 µs, total: 37.8 ms
Wall time: 36.7 ms


100